# 04_07: SOLUTION - Evaluating your GraphRAG Pipeline

In [1]:
import os
from dotenv import load_dotenv

from langchain.evaluation.qa.eval_chain import QAEvalChain
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [8]:
enhanced_graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [11]:
#examples = [
#    {"query": "What activites are included in Biathalon?",
#     "answer": "Cross-country skiing and rifle marksmanship"},
#    {"query": "What year was the International Olympic Committee founded in?", 
#     "answer": "1894"},
#    {"query": "What is a ski boot fixed to?", 
#     "answer": "Binding"},
#]

examples = [
    {"query": "What sports is the International Ski And Snowboard Federation responsible for?",
     "answer": "Alpine Skiing, Freestyle Skiing, Snowboarding, Nordic Combined, Ski Jumping, Cross-Country Skiing"},
    {"query": "What activity are ski poles not used in?", 
     "answer": "Ski jumping"},
    {"query": "Who do athletes get help from?",
     "answer": "Coaches, Peer Mentors, and Sports Psychologists"},
]



In [12]:
graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
)

In [13]:
def evaluate_graph_rag(graph_chain, examples):

    # Generate predictions by querying the graph
    predictions = []
    for ex in examples:
        graph_response = graph_chain.invoke({"query": ex["query"]})
        predictions.append({"result": graph_response["result"].strip()})

    # Run evaluation
    eval_chain = QAEvalChain.from_llm(llm)
    results = eval_chain.evaluate(examples, predictions)

    # Print output
    correct = 0
    for i, res in enumerate(results):
        print(f"Query: {examples[i]['query']}")
        print(f"Prediction from graph: {predictions[i]['result']}")
        print(f"Gold answer: {examples[i]['answer']}")
        print(f"Grade: {res['results']}")
        print("---")
        if res["results"] == "CORRECT":
            correct += 1

    accuracy = correct / len(examples)
    print(f"Graph QA Accuracy: {accuracy:.2f}")

In [14]:
evaluate_graph_rag(graph_chain, examples)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (o:Organization {id: "International Ski And Snowboard Federation"})-[:RESPONSIBLE_FOR]->(s:Sport)
RETURN s.id

Full Context:
[{'s.id': 'Cross-Country Skiing'}, {'s.id': 'Alpine Skiing'}, {'s.id': 'Ski Jumping'}, {'s.id': 'Nordic Combined'}, {'s.id': 'Freestyle Skiing'}, {'s.id': 'Snowboarding'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Activity)
WHERE NOT (a)-[:WITHOUT_USE_OF]->(:Equipment {id: "Ski Poles"})
RETURN a.id

Full Context:
[{'a.id': 'Skiing'}, {'a.id': 'Alpine Skiing'}, {'a.id': 'Nordic Skiing'}, {'a.id': 'Telemark Skiing'}, {'a.id': 'Cross-Country Skiing'}, {'a.id': 'Ski Jumping'}, {'a.id': 'Downhill Skiing'}, {'a.id': 'Off-Piste Skiing'}, {'a.id': 'Back-Country Skiing'}, {'a.id': 'Sport'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:System)-[:HELP]->(p:Person)
RETURN s.id AS System, p.id 

In [20]:
examples = [
    {"query": "What training method does the International Ski and Snowboard Federation use?",
     "answer": "Grass skiing"},
    {"query": "How does research play a role?",
     "answer": "Research improves competition by looking at Surface Science and Friction Reduction and has implications for Competitive Performance and Recreational Enjoyment"},
    {"query": "What country was the women's overall winner of the 2024–25 Fis Alpine Ski World Cup from?",
     "answer": "Italy."},
]

In [22]:
evaluate_graph_rag(graph_chain, examples)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (o:Organization {id: "International Ski and Snowboard Federation"})-[:GOVERNED_BY]->(d:Document)-[:MENTIONS]->(s:Sport)-[:TRAINING_METHOD]->(tm:Sport)
RETURN tm.id

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Concept)-[:ROLE]->(r:Concept)
WHERE c.id = "Research"
RETURN r

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:WOMEN_OVERALL_WINNER]->(e:Event), (p)-[:REPRESENTS]->(c:Country)
WHERE e.id = "2024–25 Fis Alpine Ski World Cup"
RETURN c.id AS Country

Full Context:
[{'Country': 'Italy'}]

> Finished chain.
Query: What training method does the International Ski and Snowboard Federation use?
Prediction from graph: I don't know the answer.
Gold answer: Grass skiing
Grade: INCORRECT
---
Query: How does research play a role?
Prediction from graph: I don't know the answ

In [23]:
CYPHER_GENERATION_TEMPLATE = """

You are an expert Neo4j developer. Your job is to generate a Cypher query that will traverse exactly 1–3 hops out from your focus node.

Steps:

1. Identify the main entity in the question and Capitalize Multi-Word IDs
2. Write a variable-length MATCH using the new syntax:
   MATCH path = (a {{id: "{{entity}}"}})-[r]-{{1,3}}(b)
   RETURN path;

Only return the Cypher query.

Schema:
{schema}

Question:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

enhanced_graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [24]:
evaluate_graph_rag(enhanced_graph_chain, examples)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH path = (a {id: "International Ski And Snowboard Federation"})-[r]-{1,3}(b)
RETURN path;

Full Context:
[{'path': [{'id': 'International Ski And Snowboard Federation'}, 'RECOGNIZED_SKI_FLYING', {'id': '1938'}]}, {'path': [{'id': 'International Ski And Snowboard Federation'}, 'RECOGNIZED_SKI_FLYING', {'id': '1938'}, 'BEFORE', {'id': 'World War Ii'}]}, {'path': [{'id': 'International Ski And Snowboard Federation'}, 'RECOGNIZED_SKI_FLYING', {'id': '1938'}, 'BEFORE', {'id': 'World War Ii'}, 'MENTIONS', {'summary': 'The International Ski and Snowboard Federation, also known as FIS (French: Fédération Internationale de Ski et de Snowboard), is the highest international governing body for skiing and snowboarding. It was previously known as the International Ski Federation (Fédération Internationale de Ski) until 26 May 2022 when the name was changed to include snowboard.\nFounded on 2 February 1924 in Chamonix, France 

In [25]:
ANSWER_SYNTHESIS_TEMPLATE = """
You have a list of variable‑length paths from Neo4j in the form:

(path = [(a)-[:REL1]->(X)-[:REL2]->(b), …])

Your job: extract the intermediate node X and the two relationship names REL1 and REL2,
then produce one sentence of the form:

“The {a.id} {rel1_verb} on {X.id} that {rel2_verb} {b.id}.”

Example:

Paths:
1. (Skiing Community)-[INNOVATE]-(Techniques)-[ENHANCE]-(Safety)

Answer:
The Skiing Community innovates on Techniques that enhance Safety.

Now, given:
{paths}

Answer:
"""

answer_prompt = PromptTemplate(
    template=ANSWER_SYNTHESIS_TEMPLATE,
    input_variables=["paths"],
)

full_graph_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=enhanced_graph,
    cypher_prompt=cypher_generation_prompt,
    answer_prompt=answer_prompt,      # ← override the summarizer
    verbose=True,
    allow_dangerous_requests=True,
)

In [26]:
evaluate_graph_rag(full_graph_chain, examples)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH path = (a {id: "International Ski and Snowboard Federation"})-[r]-{1,3}(b)
RETURN path;

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH path = (a {id: "Research"})-[r]-{1,3}(b)
RETURN path;

Full Context:
[{'path': [{'id': 'Research'}, 'IMPLICATIONS_FOR', {'id': 'Competitive Performance'}]}, {'path': [{'id': 'Research'}, 'IMPLICATIONS_FOR', {'id': 'Recreational Enjoyment'}]}, {'path': [{'id': 'Research'}, 'AREA_OF', {'id': 'Surface Science'}]}, {'path': [{'id': 'Research'}, 'AREA_OF', {'id': 'Surface Science'}, 'INTERPLAYS_WITH', {'id': 'Friction Reduction'}]}, {'path': [{'id': 'Research'}, 'AREA_OF', {'id': 'Surface Science'}, 'INTERPLAYS_WITH', {'id': 'Friction Reduction'}, 'AREA_OF', {'id': 'Research'}]}, {'path': [{'id': 'Research'}, 'AREA_OF', {'id': 'Surface Science'}, 'INTERPLAYS_WITH', {'id': 'Friction Reduction'}, 'CRITICAL_IN', {'id': 'C